In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

2023-12-11 14:25:44.001073: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-11 14:25:44.148093: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-11 14:25:44.148137: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-11 14:25:44.148180: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-11 14:25:44.194542: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-11 14:25:44.195141: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [ ]:
csv = pd.read_csv('datasets/Iowa_Liquor_Sales.csv')

In [1]:
def plot_deezz_long(x,y):
    plt.figure(figsize=(100,20))
    plt.plot(x,y)

def normalizer(dataframe):

    return (dataframe - np.min(dataframe)) / (np.max(dataframe) - np.min(dataframe) )
    # return tf.keras.utils.normalize(dataframe)
    # return (dataframe - dataframe.min()) / (dataframe.max() - dataframe.min() )  

In [ ]:
def make_model(inputsize,):
    model = tf.keras.layers.LSTM(inputsize)
    return model 

In [ ]:
df = pd.read_csv('datasets/newlongIowaLiqour.csv')

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
df.dtypes

In [ ]:
df = df.drop(columns='Unnamed: 0') 
#new loaded data

In [ ]:
df.head(10)

In [ ]:
mindate, maxdate = df.Date.min() - pd.Timedelta(days=1) ,df.Date.max()
mindate, maxdate

In [ ]:
def get_sales_per_day():
    print(mindate,maxdate)
    current_date = mindate
    sum = []
    dates = []
    while current_date <= maxdate:
        dates.append(current_date)
        sum.append(
            np.sum(df['Sale (Dollars)'][current_date == df['Date']])
        )
        current_date = current_date + pd.Timedelta(days=1)
        print(current_date)
    ndf = pd.DataFrame(
        data={
            'date' : dates,
            'sales' : sum
        }
    )
    return ndf

In [ ]:
ndf = pd.read_csv('datasets/salesperdayIowaLiqour.csv')
ndf = ndf.drop(columns=['Unnamed: 0'])
ndf['date'] = pd.to_datetime(ndf['date'])

In [ ]:
ndf.head(10)

In [ ]:
plot_deezz_long(ndf['date'],ndf['sales'])

In [ ]:
df['Category Name'].value_counts()

In [ ]:
dfsorted = df.sort_values('Date')


In [ ]:
dfsorted.head()

In [ ]:
dfsorted[dfsorted['Date'] == '2012-01-03'].shape,dfsorted[dfsorted['Date'] == '2012-01-04'].shape

In [ ]:
mindate,maxdate

In [ ]:
def get_length_per_date():
    current_date = mindate
    d =  dfsorted.value_counts('Date')
    ndataframe = pd.DataFrame(
        {
            'dates': d.keys(),
            'length' : d.values
        }
    )
    lofdates = ndataframe['dates'].tolist()
    while current_date <= maxdate:
        if current_date not in lofdates:
            ndataframe.loc[len(ndataframe)] = [current_date,0]
            print(current_date)
        current_date += pd.Timedelta(days=1)
    return ndataframe.sort_values('dates').reset_index(drop=True)


In [ ]:
getlenperdate = get_length_per_date()
getlenperdate.head()

In [ ]:
def showPlot_per_year():
    current_year = mindate.year
    while current_year <= maxdate.year:
        plt.figure(figsize=(30,10))
        plt.title(str(current_year))
        plt.plot(getlenperdate.set_index('dates').loc[str(current_year)])
        current_year += 1


In [ ]:
showPlot_per_year()

In [ ]:
def similarities_per_year(fromyear,toyear):
    d = pd.DataFrame(
        columns=[
            str(i) for i in range(fromyear,toyear+1)
        ]
    )
    
    for i in range(fromyear,toyear+1):
        d[str(i)] = getlenperdate.set_index('dates').loc[str(i)].reset_index(drop=True)    

    return d.fillna(0)

In [ ]:
similarities_per_year(2012,2013).plot(figsize=(200,10),legend=True)

In [ ]:
df.head(10)

In [ ]:
# tables
# date, list of items sold present, total items sold present, total sales present
def get_these_tables():
    newdataframe = pd.DataFrame(
        columns=[
            'date',                         #date
            'list of items sold present',   #[list of category name]
            'total items sold present',     #total items sold in a day
            'total sales present',          #total sales in a day
            # *list(df['Category Name'].value_counts().keys().sort_values())
        ]
    )
    current_date = mindate
    while current_date <= maxdate:
        now =  df[df['Date'] == current_date]
        lnames = now['Category Name']
        newdataframe.loc[len(newdataframe.index)] = [
            current_date,
            list(set(lnames)),
            len(lnames),
            np.sum(now['Sale (Dollars)'])
        ]
        print(current_date)
        current_date = current_date + pd.Timedelta(days=1)
    return newdataframe
    
    

In [ ]:
# tbased = get_these_tables()
tbased = pd.read_csv('datasets/saveprogress-getthesetables.csv',).drop(columns=['Unnamed: 0'])


In [ ]:
# tbased.to_csv('saveprogress-getthesetables.csv')

In [ ]:
tbased.head(30)

In [440]:
def create_classifysales():
    newdf =  pd.DataFrame(
        columns=[
            'date',
            *list(df['Category Name'].value_counts().keys().sort_values())
        ]
    )
    newdf['date'] = tbased['date']
    newdf.set_index('date',inplace=True)

    for day in newdf.index:
        now = df[df['Date'] == day]
        cur_row = now['Category Name'].value_counts()
        if not len(cur_row.keys()) == 0:
            for key in cur_row.keys():
                newdf.loc[day][key] = cur_row[key]

    return newdf

# df[df['Date'] == mindate + pd.Timedelta(days=1) ]['Category Name'].value_counts()['VODKA 80 PROOF']
# classifysales =  create_classifysales()
classifysales = pd.read_csv('datasets/saveprogress-classifysales-foreach-category.csv')
# classifysales.set_index('date',inplace=True)
classifysales.drop(columns=['date'],inplace=True)
# print(len(classifysales))
# for i in classifysales.index:
#     if classifysales.loc[i].sum() == 0  :
#         classifysales.drop(index=i,inplace=True)


In [441]:
classifysales.reset_index(drop=True,inplace=True)

In [443]:
classifysales.fillna(0,inplace=True)

In [444]:
# classifysales.to_csv('saveprogress-classifysales-foreach-category.csv')
classifysales.reset_index(drop=True,inplace=True)


In [446]:
for i in classifysales.columns:
    classifysales[str(i)] = classifysales[str(i)].astype('float')

classifysales.dtypes

100 PROOF VODKA          float64
100% AGAVE TEQUILA       float64
AGED DARK RUM            float64
AMARETTO - IMPORTED      float64
AMERICAN ALCOHOL         float64
                          ...   
WATERMELON SCHNAPPS      float64
WHISKEY LIQUEUR          float64
WHITE CREME DE CACAO     float64
WHITE CREME DE MENTHE    float64
WHITE RUM                float64
Length: 103, dtype: object

In [338]:
d = pd.read_csv('datasets/saveprogress-classifysales-foreach-category.csv').drop(columns='date')
np.max(d)
l = [20,100,50,30,10, 3]
c = (l - np.min(l)) / (np.max(l) - np.min(l) ) 
c = c * 97
c + 3

array([ 20., 100.,  50.,  30.,  10.,   3.])

In [447]:
classifysales

,100 PROOF VODKA,100% AGAVE TEQUILA,AGED DARK RUM,AMARETTO - IMPORTED,AMERICAN ALCOHOL,AMERICAN AMARETTO,AMERICAN BRANDIES,AMERICAN COCKTAILS,AMERICAN CORDIALS & LIQUEURS,AMERICAN DISTILLED SPIRITS SPECIALTY,...,TEQUILA,TRIPLE SEC,TROPICAL FRUIT SCHNAPPS,VODKA 80 PROOF,VODKA FLAVORED,WATERMELON SCHNAPPS,WHISKEY LIQUEUR,WHITE CREME DE CACAO,WHITE CREME DE MENTHE,WHITE RUM
0,110.0,0.0,0.0,0.0,51.0,94.0,0.0,422.0,0.0,0.0,...,428.0,62.0,33.0,1038.0,463.0,35.0,232.0,26.0,1.0,0.0
1,89.0,0.0,0.0,0.0,46.0,78.0,0.0,459.0,0.0,0.0,...,403.0,52.0,39.0,901.0,506.0,36.0,231.0,17.0,1.0,0.0
2,108.0,0.0,0.0,0.0,28.0,63.0,0.0,375.0,0.0,0.0,...,436.0,49.0,10.0,963.0,427.0,26.0,208.0,11.0,2.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4255,0.0,300.0,25.0,0.0,0.0,0.0,67.0,0.0,120.0,20.0,...,0.0,21.0,0.0,0.0,0.0,0.0,323.0,0.0,0.0,85.0
4256,0.0,731.0,42.0,0.0,0.0,0.0,344.0,0.0,296.0,43.0,...,0.0,38.0,0.0,0.0,0.0,0.0,962.0,0.0,0.0,223.0
4257,0.0,432.0,37.0,0.0,0.0,0.0,145.0,0.0,204.0,39.0,...,0.0,34.0,0.0,0.0,0.0,0.0,503.0,0.0,0.0,169.0


In [340]:
classifysales = normalizer(classifysales)
classifysales

,100 PROOF VODKA,100% AGAVE TEQUILA,AGED DARK RUM,AMARETTO - IMPORTED,AMERICAN ALCOHOL,AMERICAN AMARETTO,AMERICAN BRANDIES,AMERICAN COCKTAILS,AMERICAN CORDIALS & LIQUEURS,AMERICAN DISTILLED SPIRITS SPECIALTY,...,TEQUILA,TRIPLE SEC,TROPICAL FRUIT SCHNAPPS,VODKA 80 PROOF,VODKA FLAVORED,WATERMELON SCHNAPPS,WHISKEY LIQUEUR,WHITE CREME DE CACAO,WHITE CREME DE MENTHE,WHITE RUM
0,0.042835,0.000000,0.000000,0.0,0.019860,0.036604,0.000000,0.164330,0.000000,0.000000,...,0.166667,0.024143,0.012850,0.404206,0.180296,0.013629,0.090343,0.010125,0.000389,0.000000
1,0.034657,0.000000,0.000000,0.0,0.017913,0.030374,0.000000,0.178738,0.000000,0.000000,...,0.156931,0.020249,0.015187,0.350857,0.197040,0.014019,0.089953,0.006620,0.000389,0.000000
2,0.042056,0.000000,0.000000,0.0,0.010903,0.024533,0.000000,0.146028,0.000000,0.000000,...,0.169782,0.019081,0.003894,0.375000,0.166277,0.010125,0.080997,0.004283,0.000779,0.000000
3,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4254,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4255,0.000000,0.116822,0.009735,0.0,0.000000,0.000000,0.026090,0.000000,0.046729,0.007788,...,0.000000,0.008178,0.000000,0.000000,0.000000,0.000000,0.125779,0.000000,0.000000,0.033100
4256,0.000000,0.284657,0.016355,0.0,0.000000,0.000000,0.133956,0.000000,0.115265,0.016745,...,0.000000,0.014798,0.000000,0.000000,0.000000,0.000000,0.374611,0.000000,0.000000,0.086838
4257,0.000000,0.168224,0.014408,0.0,0.000000,0.000000,0.056464,0.000000,0.079439,0.015187,...,0.000000,0.013240,0.000000,0.000000,0.000000,0.000000,0.195872,0.000000,0.000000,0.065810


In [448]:
classifysales_Label = classifysales.shift(-1)

In [449]:
classifysales

,100 PROOF VODKA,100% AGAVE TEQUILA,AGED DARK RUM,AMARETTO - IMPORTED,AMERICAN ALCOHOL,AMERICAN AMARETTO,AMERICAN BRANDIES,AMERICAN COCKTAILS,AMERICAN CORDIALS & LIQUEURS,AMERICAN DISTILLED SPIRITS SPECIALTY,...,TEQUILA,TRIPLE SEC,TROPICAL FRUIT SCHNAPPS,VODKA 80 PROOF,VODKA FLAVORED,WATERMELON SCHNAPPS,WHISKEY LIQUEUR,WHITE CREME DE CACAO,WHITE CREME DE MENTHE,WHITE RUM
0,110.0,0.0,0.0,0.0,51.0,94.0,0.0,422.0,0.0,0.0,...,428.0,62.0,33.0,1038.0,463.0,35.0,232.0,26.0,1.0,0.0
1,89.0,0.0,0.0,0.0,46.0,78.0,0.0,459.0,0.0,0.0,...,403.0,52.0,39.0,901.0,506.0,36.0,231.0,17.0,1.0,0.0
2,108.0,0.0,0.0,0.0,28.0,63.0,0.0,375.0,0.0,0.0,...,436.0,49.0,10.0,963.0,427.0,26.0,208.0,11.0,2.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4255,0.0,300.0,25.0,0.0,0.0,0.0,67.0,0.0,120.0,20.0,...,0.0,21.0,0.0,0.0,0.0,0.0,323.0,0.0,0.0,85.0
4256,0.0,731.0,42.0,0.0,0.0,0.0,344.0,0.0,296.0,43.0,...,0.0,38.0,0.0,0.0,0.0,0.0,962.0,0.0,0.0,223.0
4257,0.0,432.0,37.0,0.0,0.0,0.0,145.0,0.0,204.0,39.0,...,0.0,34.0,0.0,0.0,0.0,0.0,503.0,0.0,0.0,169.0


In [450]:
classifysales_Label

,100 PROOF VODKA,100% AGAVE TEQUILA,AGED DARK RUM,AMARETTO - IMPORTED,AMERICAN ALCOHOL,AMERICAN AMARETTO,AMERICAN BRANDIES,AMERICAN COCKTAILS,AMERICAN CORDIALS & LIQUEURS,AMERICAN DISTILLED SPIRITS SPECIALTY,...,TEQUILA,TRIPLE SEC,TROPICAL FRUIT SCHNAPPS,VODKA 80 PROOF,VODKA FLAVORED,WATERMELON SCHNAPPS,WHISKEY LIQUEUR,WHITE CREME DE CACAO,WHITE CREME DE MENTHE,WHITE RUM
0,89.0,0.0,0.0,0.0,46.0,78.0,0.0,459.0,0.0,0.0,...,403.0,52.0,39.0,901.0,506.0,36.0,231.0,17.0,1.0,0.0
1,108.0,0.0,0.0,0.0,28.0,63.0,0.0,375.0,0.0,0.0,...,436.0,49.0,10.0,963.0,427.0,26.0,208.0,11.0,2.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4254,0.0,300.0,25.0,0.0,0.0,0.0,67.0,0.0,120.0,20.0,...,0.0,21.0,0.0,0.0,0.0,0.0,323.0,0.0,0.0,85.0
4255,0.0,731.0,42.0,0.0,0.0,0.0,344.0,0.0,296.0,43.0,...,0.0,38.0,0.0,0.0,0.0,0.0,962.0,0.0,0.0,223.0
4256,0.0,432.0,37.0,0.0,0.0,0.0,145.0,0.0,204.0,39.0,...,0.0,34.0,0.0,0.0,0.0,0.0,503.0,0.0,0.0,169.0
4257,0.0,598.0,33.0,0.0,0.0,0.0,218.0,0.0,253.0,30.0,...,0.0,45.0,0.0,0.0,0.0,0.0,652.0,0.0,0.0,206.0


In [451]:
classifysales_Label.dtypes

100 PROOF VODKA          float64
100% AGAVE TEQUILA       float64
AGED DARK RUM            float64
AMARETTO - IMPORTED      float64
AMERICAN ALCOHOL         float64
                          ...   
WATERMELON SCHNAPPS      float64
WHISKEY LIQUEUR          float64
WHITE CREME DE CACAO     float64
WHITE CREME DE MENTHE    float64
WHITE RUM                float64
Length: 103, dtype: object

In [452]:
classifysales.tail()

,100 PROOF VODKA,100% AGAVE TEQUILA,AGED DARK RUM,AMARETTO - IMPORTED,AMERICAN ALCOHOL,AMERICAN AMARETTO,AMERICAN BRANDIES,AMERICAN COCKTAILS,AMERICAN CORDIALS & LIQUEURS,AMERICAN DISTILLED SPIRITS SPECIALTY,...,TEQUILA,TRIPLE SEC,TROPICAL FRUIT SCHNAPPS,VODKA 80 PROOF,VODKA FLAVORED,WATERMELON SCHNAPPS,WHISKEY LIQUEUR,WHITE CREME DE CACAO,WHITE CREME DE MENTHE,WHITE RUM
4254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4255,0.0,300.0,25.0,0.0,0.0,0.0,67.0,0.0,120.0,20.0,...,0.0,21.0,0.0,0.0,0.0,0.0,323.0,0.0,0.0,85.0
4256,0.0,731.0,42.0,0.0,0.0,0.0,344.0,0.0,296.0,43.0,...,0.0,38.0,0.0,0.0,0.0,0.0,962.0,0.0,0.0,223.0
4257,0.0,432.0,37.0,0.0,0.0,0.0,145.0,0.0,204.0,39.0,...,0.0,34.0,0.0,0.0,0.0,0.0,503.0,0.0,0.0,169.0
4258,0.0,598.0,33.0,0.0,0.0,0.0,218.0,0.0,253.0,30.0,...,0.0,45.0,0.0,0.0,0.0,0.0,652.0,0.0,0.0,206.0


In [453]:
classifysales_Label.dropna(inplace=True)
classifysales_Label.tail()

,100 PROOF VODKA,100% AGAVE TEQUILA,AGED DARK RUM,AMARETTO - IMPORTED,AMERICAN ALCOHOL,AMERICAN AMARETTO,AMERICAN BRANDIES,AMERICAN COCKTAILS,AMERICAN CORDIALS & LIQUEURS,AMERICAN DISTILLED SPIRITS SPECIALTY,...,TEQUILA,TRIPLE SEC,TROPICAL FRUIT SCHNAPPS,VODKA 80 PROOF,VODKA FLAVORED,WATERMELON SCHNAPPS,WHISKEY LIQUEUR,WHITE CREME DE CACAO,WHITE CREME DE MENTHE,WHITE RUM
4253,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4254,0.0,300.0,25.0,0.0,0.0,0.0,67.0,0.0,120.0,20.0,...,0.0,21.0,0.0,0.0,0.0,0.0,323.0,0.0,0.0,85.0
4255,0.0,731.0,42.0,0.0,0.0,0.0,344.0,0.0,296.0,43.0,...,0.0,38.0,0.0,0.0,0.0,0.0,962.0,0.0,0.0,223.0
4256,0.0,432.0,37.0,0.0,0.0,0.0,145.0,0.0,204.0,39.0,...,0.0,34.0,0.0,0.0,0.0,0.0,503.0,0.0,0.0,169.0
4257,0.0,598.0,33.0,0.0,0.0,0.0,218.0,0.0,253.0,30.0,...,0.0,45.0,0.0,0.0,0.0,0.0,652.0,0.0,0.0,206.0


In [454]:
classifysales.shape,classifysales_Label.shape

((4259, 103), (4258, 103))

In [455]:
classifysales = classifysales.iloc[:-1,:]

In [456]:
classifysales.shape,classifysales_Label.shape

((4258, 103), (4258, 103))

In [457]:
classifysales.tail(10)

,100 PROOF VODKA,100% AGAVE TEQUILA,AGED DARK RUM,AMARETTO - IMPORTED,AMERICAN ALCOHOL,AMERICAN AMARETTO,AMERICAN BRANDIES,AMERICAN COCKTAILS,AMERICAN CORDIALS & LIQUEURS,AMERICAN DISTILLED SPIRITS SPECIALTY,...,TEQUILA,TRIPLE SEC,TROPICAL FRUIT SCHNAPPS,VODKA 80 PROOF,VODKA FLAVORED,WATERMELON SCHNAPPS,WHISKEY LIQUEUR,WHITE CREME DE CACAO,WHITE CREME DE MENTHE,WHITE RUM
4248,0.0,433.0,27.0,0.0,0.0,0.0,219.0,0.0,203.0,30.0,...,0.0,22.0,0.0,0.0,0.0,0.0,654.0,0.0,0.0,151.0
4249,0.0,481.0,20.0,0.0,0.0,0.0,238.0,0.0,234.0,34.0,...,0.0,26.0,0.0,0.0,0.0,0.0,648.0,0.0,0.0,161.0
4250,0.0,467.0,29.0,0.0,0.0,0.0,178.0,0.0,231.0,50.0,...,0.0,47.0,0.0,0.0,0.0,0.0,605.0,0.0,0.0,192.0
4251,0.0,432.0,21.0,0.0,0.0,0.0,165.0,0.0,199.0,29.0,...,0.0,31.0,0.0,0.0,0.0,0.0,438.0,0.0,0.0,145.0
4252,0.0,513.0,22.0,0.0,0.0,0.0,259.0,0.0,300.0,41.0,...,0.0,34.0,0.0,0.0,0.0,0.0,921.0,0.0,0.0,239.0
4253,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4255,0.0,300.0,25.0,0.0,0.0,0.0,67.0,0.0,120.0,20.0,...,0.0,21.0,0.0,0.0,0.0,0.0,323.0,0.0,0.0,85.0
4256,0.0,731.0,42.0,0.0,0.0,0.0,344.0,0.0,296.0,43.0,...,0.0,38.0,0.0,0.0,0.0,0.0,962.0,0.0,0.0,223.0
4257,0.0,432.0,37.0,0.0,0.0,0.0,145.0,0.0,204.0,39.0,...,0.0,34.0,0.0,0.0,0.0,0.0,503.0,0.0,0.0,169.0


In [458]:
classifysales_Label.isnull().any()

100 PROOF VODKA          False
100% AGAVE TEQUILA       False
AGED DARK RUM            False
AMARETTO - IMPORTED      False
AMERICAN ALCOHOL         False
                         ...  
WATERMELON SCHNAPPS      False
WHISKEY LIQUEUR          False
WHITE CREME DE CACAO     False
WHITE CREME DE MENTHE    False
WHITE RUM                False
Length: 103, dtype: bool

In [459]:
target = int(len(classifysales) * 0.8)
xtrain = classifysales[ : target]
ytrain = classifysales_Label[ : target]

xtest = classifysales[target :]
ytest = classifysales_Label[target :]

In [498]:
model = tf.keras.Sequential([
        tf.keras.layers.Input(classifysales.columns.__len__()),
        tf.keras.layers.Dense(103*15, activation='relu'),
        # tf.keras.layers.LeakyReLU(alpha=0.01),#512
        tf.keras.layers.Dense(classifysales.columns.__len__(),activation='relu')
    ],)

model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss=tf.keras.losses.MeanAbsoluteError(),
        metrics=tf.keras.metrics.MeanAbsolutePercentageError(),

)
    


In [499]:
model.fit(xtrain.to_numpy() ,ytrain.to_numpy(),epochs=100,shuffle=False,)

Epoch 1/100
107/107 [==============================] - 1s 3ms/step - loss: 51.9216 - mean_absolute_percentage_error: 2979112704.0000
Epoch 2/100
107/107 [==============================] - 0s 3ms/step - loss: 53.2548 - mean_absolute_percentage_error: 1817988992.0000
Epoch 3/100
107/107 [==============================] - 0s 3ms/step - loss: 52.9424 - mean_absolute_percentage_error: 1497138688.0000
Epoch 4/100
107/107 [==============================] - 0s 3ms/step - loss: 51.9520 - mean_absolute_percentage_error: 1762223232.0000
Epoch 5/100
107/107 [==============================] - 0s 3ms/step - loss: 50.6932 - mean_absolute_percentage_error: 1678928000.0000
Epoch 6/100
107/107 [==============================] - 0s 3ms/step - loss: 50.3055 - mean_absolute_percentage_error: 1538813184.0000
Epoch 7/100
107/107 [==============================] - 0s 3ms/step - loss: 49.8827 - mean_absolute_percentage_error: 1452463872.0000
Epoch 8/100
107/107 [==============================] - 0s 3ms/step - 

In [500]:
model.summary()

Model: "sequential_77"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_165 (Dense)           (None, 1545)              160680    
                                                                 
 dense_166 (Dense)           (None, 103)               159238    
                                                                 
Total params: 319918 (1.22 MB)
Trainable params: 319918 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [501]:
p = 4
res = model.predict(xtest.to_numpy()[p].reshape(1,len(classifysales.columns)))
# classifysales_Label.to_numpy()[5]

print((res[0] * np.max(d)).sum() )
print((classifysales.loc[p] * np.max(d)).sum() )

# l = [
#    model.predict(classifysales.to_numpy()[i].reshape(1,len(classifysales.columns)))[0] for i in range(100)
# ]

print('{:<40} {:<40} {:<40}'.format('predicted','actual','input'))
for i in range(0,len(res[0])):
    # print('{:<40} {:<40} {:<40}'.format(res[0][i] * np.max(d)    , ytest.to_numpy()[p][i]* np.max(d) ,xtest.to_numpy()[p][i]* np.max(d)))
    print('{:<40} {:<40} {:<40}'.format(res[0][i] , ytest.to_numpy()[p][i] ,xtest.to_numpy()[p][i]))



1/1 [==============================] - 0s 30ms/step
11141894.0
0.0
predicted                                actual                                   input                                   
0.0                                      0.0                                      0.0                                     
286.60858154296875                       353.0                                    328.0                                   
0.0                                      48.0                                     42.0                                    
0.0                                      0.0                                      0.0                                     
0.0                                      0.0                                      0.0                                     
0.0                                      0.0                                      0.0                                     
0.0                                      209.0                          

In [497]:
(model.evaluate(xtest,ytest)[0] -1) * 100  

27/27 [==============================] - 0s 2ms/step - loss: 40.5512 - mean_absolute_error: 40.5512


3955.1239013671875

In [359]:
classifysales[:3]

,100 PROOF VODKA,100% AGAVE TEQUILA,AGED DARK RUM,AMARETTO - IMPORTED,AMERICAN ALCOHOL,AMERICAN AMARETTO,AMERICAN BRANDIES,AMERICAN COCKTAILS,AMERICAN CORDIALS & LIQUEURS,AMERICAN DISTILLED SPIRITS SPECIALTY,...,TEQUILA,TRIPLE SEC,TROPICAL FRUIT SCHNAPPS,VODKA 80 PROOF,VODKA FLAVORED,WATERMELON SCHNAPPS,WHISKEY LIQUEUR,WHITE CREME DE CACAO,WHITE CREME DE MENTHE,WHITE RUM
0,0.042835,0.0,0.0,0.0,0.019860,0.036604,0.0,0.164330,0.0,0.0,...,0.166667,0.024143,0.012850,0.404206,0.180296,0.013629,0.090343,0.010125,0.000389,0.0
1,0.034657,0.0,0.0,0.0,0.017913,0.030374,0.0,0.178738,0.0,0.0,...,0.156931,0.020249,0.015187,0.350857,0.197040,0.014019,0.089953,0.006620,0.000389,0.0
2,0.042056,0.0,0.0,0.0,0.010903,0.024533,0.0,0.146028,0.0,0.0,...,0.169782,0.019081,0.003894,0.375000,0.166277,0.010125,0.080997,0.004283,0.000779,0.0


In [360]:
classifysales_Label[:3]

,100 PROOF VODKA,100% AGAVE TEQUILA,AGED DARK RUM,AMARETTO - IMPORTED,AMERICAN ALCOHOL,AMERICAN AMARETTO,AMERICAN BRANDIES,AMERICAN COCKTAILS,AMERICAN CORDIALS & LIQUEURS,AMERICAN DISTILLED SPIRITS SPECIALTY,...,TEQUILA,TRIPLE SEC,TROPICAL FRUIT SCHNAPPS,VODKA 80 PROOF,VODKA FLAVORED,WATERMELON SCHNAPPS,WHISKEY LIQUEUR,WHITE CREME DE CACAO,WHITE CREME DE MENTHE,WHITE RUM
0,0.034657,0.0,0.0,0.0,0.017913,0.030374,0.0,0.178738,0.0,0.0,...,0.156931,0.020249,0.015187,0.350857,0.197040,0.014019,0.089953,0.006620,0.000389,0.0
1,0.042056,0.0,0.0,0.0,0.010903,0.024533,0.0,0.146028,0.0,0.0,...,0.169782,0.019081,0.003894,0.375000,0.166277,0.010125,0.080997,0.004283,0.000779,0.0
2,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [361]:
model.evaluate(classifysales,classifysales_Label,) 

  1/134 [..............................] - ETA: 2s - loss: 36.1933 - mean_absolute_percentage_error: 36.1933

134/134 [==============================] - 0s 2ms/step - loss: 33.7325 - mean_absolute_percentage_error: 33.7325


[33.73252487182617, 33.73252487182617]

In [362]:
# model.save('forecasting_model-per-item.keras')

In [363]:
# conveter = tf.lite.TFLiteConverter.from_keras_model(model)
# tflite_model = conveter.convert()
# 
# Save the model.
# with open('forecasting_model-per-item.tflite', 'wb') as f:
    # f.write(tflite_model)